Steps:
    
Sample patches from DIV2K 
Calculate the bicubic psnr of each patch 

Calculate entropy of gradients of each grayscaled patch 

Plot entropy vs. psnr 
Fit the line



In [ ]:
import os
import glob
import core
import torch
import matplotlib.pyplot as plt 
import torchvision.transforms as transforms
import skimage.measure    
import math

In [ ]:
## NOTE these are the corrupted LR and HR pairs of 900 images. 
## They are corrupted using 
## https://github.com/sanghyun-son/bicubic_pytorch/blob/master/core.py to do matlab like bicubic interpolation.

base_dir = "path/to/DIV2K/after/using/PRE Processing/Script"
hr_image_paths = sorted(glob.glob(os.path.join(base_dir,"DIV2K_train_HR/") + "*"))
lr_image_paths = sorted(glob.glob(os.path.join(base_dir,"DIV2K_train_LR_bicubic/X4/") + "*"))


## DIV2K Training Data is the first 800 images. Only include those.
hr_image_paths = hr_image_paths[:800]
lr_image_paths = lr_image_paths[:800]

####  Step 1: Sample LR, HR patches. 

In [ ]:
import random
import numpy as np
from PIL import Image

def sample_patches(lr_img_path, hr_img_path, patch_size, scale,num_patches):
    # Load images
    random.seed(123)
    lr_img = Image.open(lr_img_path)
    hr_img = Image.open(hr_img_path)
    
    # Convert images to numpy arrays
    lr_arr = np.array(lr_img)
    hr_arr = np.array(hr_img)
    
    # Initialize arrays to store patches
    lr_patches = np.zeros((num_patches, patch_size, patch_size, lr_arr.shape[2]))
    hr_patches = np.zeros((num_patches, patch_size*scale, patch_size*scale, hr_arr.shape[2]))
    
    # Sample patches randomly
    for i in range(num_patches):
        # Choose random location for patch
        x = random.randint(0, lr_arr.shape[0] - patch_size)
        y = random.randint(0, lr_arr.shape[1] - patch_size)
        
        # Extract patch from LR and HR images
        # Add patch to patch arrays
        lr_patches[i] = lr_arr[x:x+patch_size, y:y+patch_size, :]
        hr_patches[i] =  hr_arr[x*scale:x*scale+patch_size*scale, y*scale:y*scale+patch_size*scale, :]
    
    
    ## convert them to uint8 
    lr_patches = lr_patches.astype("uint8") 
    hr_patches = hr_patches.astype("uint8") 
    return lr_patches, hr_patches

In [ ]:
def calculate_psnr(a,b,rgb_range=255):
    diff = (a - b) / rgb_range
    mse = diff.pow(2).mean()
    if mse == 0:
        print("MSE here was 0. Math domain error!")
        return -1
    
    return -10 * math.log10(mse)

def calculate_bicubic_psnrs(lr_patches,hr_patches, patch_size,scale):
    '''given two numpy arrays of lr patches and hr patches
    use the core implementation to calculate the bicbuc psnr. 
    this requires converting to tensor first
    
    This is done in 0-255 range.
    '''
    

    vals = []
    
    for i in range(len(lr_patches)):
        
        
        upsampled = core.imresize(lr_patches[i], sizes=(patch_size*scale, patch_size*scale))        
        upsampled = upsampled.clip(0,255).int() 
        psnr = calculate_psnr(hr_patches[i],upsampled)
        vals.append(psnr)
        
    return vals


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class Gradient_Net(nn.Module):

    def __init__(self,device,pad=0):

        super(Gradient_Net, self).__init__()
        kernel_x = [[-1., 0., 1.], [-2., 0., 2.], [-1., 0., 1.]]
        kernel_x = torch.FloatTensor(kernel_x).unsqueeze(0).unsqueeze(0).to(device)
        kernel_y = [[-1., 0., 1.], [-2., 0., 2.], [-1., 0., 1.]]
        kernel_y = torch.FloatTensor(kernel_y).unsqueeze(0).unsqueeze(0).to(device)
        self.weight_x = nn.Parameter(data=kernel_x, requires_grad=False)
        self.weight_y = nn.Parameter(data=kernel_y, requires_grad=False)
        self.pad = pad

    def forward(self, x):
        grad_x = F.conv2d(x, self.weight_x,padding=self.pad)
        grad_y = F.conv2d(x, self.weight_y,padding=self.pad)
        gradient = torch.abs(grad_x) + torch.abs(grad_y)
        return gradient


def calculate_entropy_gradients(patches):
    '''
    calculates the entropy of the gradients of the grayscaled patches
    
    this calculation should be in 0-1 range
    '''
    
    patches = patches / 255.0
    
    ## Converts a torch.*Tensor of shape C x H x W or a numpy ndarray of shape H x W x C to a PIL 
    ## Image while preserving the value range.
    pre_process = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Grayscale(num_output_channels=1),
                                  transforms.ToTensor()])

    #step A: convert to grayscale first.
    gray_patches = torch.stack([pre_process(i) for i in patches])
    
    #step B: get gradients.
    gradients = gradient_net(gray_patches.to(device)).cpu().numpy()
    
    #step C: get entropy.
    entropies = [ent(e) for e in gradients]

    return entropies
    
   
    

In [ ]:
patch_size = 48
num_patches = 100  
scale = 4
counter = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gradient_net = Gradient_Net(device,1).to(device)
ent = skimage.measure.shannon_entropy

global_xs =[]
global_ys =[]

for lr_img_path, hr_img_path in zip(lr_image_paths,hr_image_paths):
    
    print(counter)
    counter+=1
    


    lr_patches, hr_patches = sample_patches(lr_img_path, hr_img_path, 
                                            patch_size,scale, num_patches)

    # bring channels first for pytorch tensor.

    lr_patches_tensor = torch.Tensor(lr_patches).permute(0,3,1,2) 
    hr_patches_tensor = torch.Tensor(hr_patches).permute(0,3,1,2) 

    ## calculate bicubic psnrs 
    bcs = calculate_bicubic_psnrs(lr_patches_tensor,hr_patches_tensor, patch_size,scale)

    ## calculate entropy of gradients 
    ents = calculate_entropy_gradients(hr_patches_tensor)


    ## save them 
    global_xs += ents 
    global_ys += bcs 
        
    

In [ ]:
xs = np.array(global_xs) 
ys = np.array(global_ys) 


# remove patches that had a psnr of -1 as the MSE there was 0. 
## or replace them actually with PSNR of max PSNR. or 1000 
## also remove super high PSNR patches / outliers  

filter_ = (np.array(ys) >= 0) & (np.array(ys) <= 60)
xs = xs[filter_]
ys = ys[filter_] 

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(10,10),sharey=True,sharex=True)


plt.scatter(xs,ys,color='black')
plt.yticks(fontsize=13 ) 
plt.xticks(fontsize=13 )     
z = np.poly1d(np.polyfit(xs,ys,1))
plt.plot(xs, z(xs),color='red')   
fig.text(0.515, 0.05, 'Entropy', ha='center',fontsize=20)
fig.text(0.05, 0.5, 'PSNR', va='center', rotation='vertical',fontsize=20)
plt.savefig("./****_entropy_vs_psnr_DIV2K.pdf")
plt.show()